<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/Preprocess_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mne matplotlib

     |████████████████████████████████| 7.5 MB 5.0 MB/s 


In [58]:
import numpy as np
import mne 
import pandas as pd
import os
import pylab as pl
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/MLTSA22/Final_Project

/content/drive/MyDrive/MLTSA22/Final_Project


In [5]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project

00000258_s002_t000.edf
all_tuh_eeg_seizure_links.txt
dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf
dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf


In [34]:
file = ['00000258_s002_t000.edf', 'dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf',
        'dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf']

In [144]:
def preprocessing(file):
  #read the data
  data = mne.io.read_raw_edf(file)
  data = data.copy().load_data()
  #notch filter 60 Hz - to get rid of the noise because of the power outlets
  eeg_picks = mne.pick_types(data.info, eeg=True)
  freqs = []
  for i in range(0, int(data.info['sfreq']/2), 60):
    if i != 0:
      freqs.append(i)
  data_notch = data.notch_filter(freqs=np.asarray(freqs), picks=eeg_picks)

  #high pass filter
  data_filter = data_notch.filter(l_freq=0.2, h_freq=None)

  #downsampling
  data_done = data_filter.resample(sfreq=200)
  return data_done

In [181]:
data_first = preprocessing(file[0])
df_first = pd.DataFrame(data_first.get_data())
df_first['channel'] = data_first.ch_names
df_first['patient_id'] = file[0][-22:-14]
for i in range(1, len(file)):
  data = preprocessing(file[i])
  df = data.get_data()
  df = pd.DataFrame(df)
  df['channel'] = data.ch_names
  df['patient_id'] = file[i][-22:-14]
  df_first = pd.concat([df_first, df])

Extracting EDF parameters from /content/drive/MyDrive/MLTSA22/Final_Project/00000258_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7999  =      0.000 ...    19.997 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB c

In [183]:
df_first.head()

,0,1,2,3,4,5,6,7,8,9,...,44790,44791,44792,44793,44794,44795,44796,44797,44798,44799
0,-4.379170e-09,-0.000010,-0.000013,-0.000010,-0.000014,-2.185444e-05,-0.000014,-0.000013,-0.000023,-0.000022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-4.454867e-09,-0.000012,-0.000016,-0.000012,-0.000019,-2.526137e-05,-0.000020,-0.000020,-0.000025,-0.000028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.859807e-08,-0.000005,-0.000004,-0.000008,-0.000006,-1.352105e-05,-0.000006,-0.000007,-0.000014,-0.000014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-8.383492e-09,-0.000004,-0.000004,0.000004,0.000003,9.815249e-07,0.000007,0.000011,0.000005,0.000004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.380791e-11,-0.000005,-0.000009,-0.000003,-0.000002,-8.697765e-06,-0.000005,-0.000005,-0.000008,-0.000007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
df_first.shape

(96, 44802)

In [185]:
df_first.columns

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       44790, 44791, 44792, 44793, 44794, 44795, 44796, 44797, 44798, 44799],
      dtype='object', length=44802)

In [180]:
#DOESN'T WORK PROPERLY
data_first = preprocessing(file[0])
df_first = pd.DataFrame(data_first.get_data())
df_first['channel'] = data_first.ch_names
df_first['patient_id'] = file[0][-22:-14]
baseline_samples = data_first.n_times
for i in range(1, len(file)):
  data = preprocessing(file[i])
  df = data.get_data()
  '''if data.n_times > baseline_samples:
    counter = 0
    for j in range(0, data.n_times, baseline_samples):
      if j != 0:
        df1 = pd.DataFrame(df[counter:j])
        print(df1.shape)
        print(len(data.ch_names))
        print(df_first.shape)
        df1['channel'] = data.ch_names
        df1['patient_id'] = file[i][-22:-14]
        df_first = pd.concat([df_first, df1])
        counter += j'''
  else:
    df = pd.DataFrame(df)
    df['channel'] = data.ch_names
    df['patient_id'] = file[i][-22:-14]
    df_first = pd.concat([df_first, df])

Extracting EDF parameters from /content/drive/MyDrive/MLTSA22/Final_Project/00000258_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7999  =      0.000 ...    19.997 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB c